In [3]:
from pys import credentials 

In [5]:
# Create Workspace

from azureml.core import Workspace

ws = Workspace.create(name                  = "handmade_ntt_data_challenge",
                      subscription_id       = credentials.azure_subscription_id,
                      resource_group        = credentials.azure_resource_group,
                      create_resource_group = False,
                      location              = credentials.azure_location)

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Deploying StorageAccount with name handmadestorage822b6e018.
Deploying AppInsights with name handmadeinsights4f9e508a.
Deployed AppInsights with name handmadeinsights4f9e508a. Took 4.13 seconds.
Deploying KeyVault with name handmadekeyvault0f212c84.
Deployed KeyVault with name handmadekeyvault0f212c84. Took 18.55 seconds.
Deploying Workspace with name handmade_ntt_data_challenge.
Deployed StorageAccount with name handmadestorage822b6e018. Took 23.62 seconds.
Deployed Workspace with name handmade_ntt_data_challenge. Took 35.61 seconds.


In [4]:
# Connect to Workspace

from azureml.core import Workspace

ws = Workspace(workspace_name        = "handmade_ntt_data_challenge",
               subscription_id       = credentials.azure_subscription_id,
               resource_group        = credentials.azure_resource_group)

In [7]:
# Register the model

from azureml.core.model import Model

model = Model.register(workspace  = ws, 
                       model_path = "../resources/model.pkl", 
                       model_name = "handmade_model")

Registering model handmade_model


In [35]:
# Register the x_scaler

from azureml.core.model import Model

model = Model.register(workspace  = ws, 
                       model_path = "../resources/x_scaler.pkl", 
                       model_name = "handmade_x_scaler")

Registering model handmade_x_scaler


In [24]:
# Define inference config

from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name = "handmade_env")
dummy_inference_config = InferenceConfig(environment      = env,
                                         source_directory = "../pys/",
                                         entry_script     = "../pys/echo_score.py")

In [23]:
# Local deployment

from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [25]:
service = Model.deploy(workspace         = ws,
                       name              = "handmade-predict",
                       models            = [model],
                       inference_config  = dummy_inference_config,
                       deployment_config = deployment_config,
                       overwrite         = True)

service.wait_for_deployment(show_output = True)

/var/folders/10/5nbnv2696vlgb605wf3g6wwh0000gn/T/ipykernel_32754/428816697.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace         = ws,


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry e4e0e96ebb3f40cdb53da54690e88f94.azurecr.io
Logging into Docker registry e4e0e96ebb3f40cdb53da54690e88f94.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM e4e0e96ebb3f40cdb53da54690e88f94.azurecr.io/azureml/azureml_de295a9a9e47e09a199d4bf845dd6bf2
 ---> 55c9e0ac853c
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 793bdb6c7040
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjQ2ZmE4NTZiLTgyYzgtNGIzNy04MzM5LTk2ZDBmMzg1OTljNyIsInJlc291cmNlR3JvdXBOYW1lIjoiZ3UxIiwiYWNjb3VudE5hbWUiOiJoYW5kbWFkZV9udHRfZGF0YV9jaGFsbGVuZ2UiLCJ3b3Jrc3BhY2VJZCI6ImU0ZTBlOTZlLWJiM2YtNDBjZC1iNTNkLWE1NDY5MGU4OGY5NCJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform w

In [97]:
# Local deployment but with model

from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="cloudenv")
python_packages = ['joblib', 'numpy', 'scikit_learn==1.0.2']

for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

inference_config = InferenceConfig(environment=env, source_directory='../pys/', entry_script='../pys/score.py')

In [98]:
from azureml.core.model import Model

model = Model(ws, "handmade_model")

service = Model.deploy(workspace         = ws,
                       name              = "handmade-predict",
                       models            = [model],
                       inference_config  = inference_config,
                       deployment_config = deployment_config,
                       overwrite         = True)

service.wait_for_deployment(show_output = True)

/var/folders/10/5nbnv2696vlgb605wf3g6wwh0000gn/T/ipykernel_32754/3685342045.py:5: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace         = ws,


Generating Docker build context.
2023/03/28 10:49:03 Downloading source code...
2023/03/28 10:49:04 Finished downloading source code
2023/03/28 10:49:04 Creating Docker network: acb_default_network, driver: 'bridge'
2023/03/28 10:49:05 Successfully set up Docker network: acb_default_network
2023/03/28 10:49:05 Setting up Docker configuration...
2023/03/28 10:49:06 Successfully set up Docker configuration
2023/03/28 10:49:06 Logging in to registry: e4e0e96ebb3f40cdb53da54690e88f94.azurecr.io
2023/03/28 10:49:06 Successfully logged into e4e0e96ebb3f40cdb53da54690e88f94.azurecr.io
2023/03/28 10:49:06 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2023/03/28 10:49:06 Scanning for dependencies...
2023/03/28 10:49:07 Successfully scanned dependencies
2023/03/28 10:49:07 Launching container with name: acb_step_0
Sending build context to Docker daemon  71.68kB
Step 1/21 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230120.

In [99]:
import requests
import json
import numpy as np
import joblib

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}

data = {
        "edad": 40.0,
        "trabajo": 40.0,
        "deuda": 40.0,
        "saldo": 40.0,
        "vivienda": 40.0,
        "prestamo": 40.0,
        "duracion": 40.0,
        "fecha_contacto": 40.0,
        "campaign": 40.0,
        "tiempo_transcurrido": 40.0,
        "contactos_anteriores": 40.0,
        "target": 40.0,
        "contactado": 40.0,
        "desconocido": 40.0,
        "fijo": 40.0,
        "movil": 40.0,
        "casado": 40.0,
        "divorciado": 40.0,
        "soltero": 40.0,
        "exito": 40.0,
        "nuevo_cliente": 40.0,
        "otro": 40.0,
        "sin_exito": 0,
        "primaria": 1,
        "secundaria/superiores": 0,
        "universitarios": 40.0
      }

input = []
scaler = joblib.load("../resources/x_scaler.pkl")

for i in data:

    input.append(data[i])

input = np.array([input])

scaled_input = scaler.transform(input)

for enum, i in enumerate(data):
    
    data[i] = scaled_input.tolist()[0][enum]


response = requests.post(uri, data=json.dumps(data), headers=headers)
print(response.json())

This client wouldn't contract the fixed deposit


In [8]:
# Cloud deployment


# Deployment config cloud

from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1, auth_enabled=True)

# Inference config tested with model and working

from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="cloudenv")
python_packages = ['joblib', 'numpy', 'scikit_learn==1.0.2']

for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

inference_config = InferenceConfig(environment=env, source_directory='../pys/', entry_script='../pys/score.py')

# Model deployment

from azureml.core.model import Model

model = Model(ws, "handmade_model")

service = Model.deploy(workspace         = ws,
                       name              = "handmade-predict",
                       models            = [model],
                       inference_config  = inference_config,
                       deployment_config = deployment_config,
                       overwrite         = True)

service.wait_for_deployment(show_output = True)

/var/folders/10/5nbnv2696vlgb605wf3g6wwh0000gn/T/ipykernel_4521/4294590921.py:29: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace         = ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-03-29 12:42:28+02:00 Creating Container Registry if not exists.
2023-03-29 12:42:28+02:00 Registering the environment.
2023-03-29 12:42:30+02:00 Use the existing image.
2023-03-29 12:42:33+02:00 Generating deployment configuration.
2023-03-29 12:42:34+02:00 Submitting deployment to compute..
2023-03-29 12:42:50+02:00 Checking the status of deployment handmade-predict..
2023-03-29 12:45:37+02:00 Checking the status of inference endpoint handmade-predict.
Succeeded
ACI service creation operation finished, operation "Succeeded"
